#### Script to check rf and compute the visuomemory index block 1
- First, check if neuron responding to the task by comparing activity during trial vs baseline
- Second, check whether the rf is in in or out
- Third, compute the visuomemory index

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.trials import align_trials
from ephysvibe.task import task_constants
from ephysvibe.structures.neuron_data import NeuronData
from scipy import stats

### Def functions

### Start analysis

In [3]:
neu_path = "/home/INT/losada.c/Documents/session_struct/v4/neurons/2023-03-03_10-59-32_Riesling_v4_e1_r1_mua7_neu.h5"
neu_data = NeuronData.from_python_hdf5(neu_path)

#### Select parameters

In [4]:
# parameters
time_before=500
select_block=2
start=-200
end=1200
idx_start = time_before + start 
idx_end = time_before+end
# ---conv
fs = 1000
w_size=0.1
w_std=0.015
# Timings 
## fixation
dur_fix = 200
## visual stim
st_v =   80
end_v = 300
## delay
st_d = 600 
end_d = 800
# trials and threshold
min_trials = 3
n_spikes = 1
p_threshold = 0.05
vd_threshold = 0.4

In [5]:
# select correct trials, block one, inside RF, and align with sample onset 
sp_sample_on_in,mask_in = align_trials.align_on(
    sp_samples=neu_data.sp_samples,
    code_samples=neu_data.code_samples,
    code_numbers=neu_data.code_numbers,
    trial_error=neu_data.trial_error,
    block=neu_data.block,
    pos_code=neu_data.pos_code,
    select_block= 1,
    select_pos= 1,
    event ="sample_on",
    time_before = time_before,
    error_type= 0,
)
# conv_in = firing_rate.convolve_signal(arr=sp_sample_on_in,fs = 1000,w_size=0.1,w_std=0.015,axis=1) 
sp_sample_on_in = sp_sample_on_in[:,idx_start:idx_end]

In [6]:
# select correct trials, block one, outside RF, and align with sample onset 
sp_sample_on_out,mask_out = align_trials.align_on(
    sp_samples=neu_data.sp_samples,
    code_samples=neu_data.code_samples,
    code_numbers=neu_data.code_numbers,
    trial_error=neu_data.trial_error,
    block=neu_data.block,
    pos_code=neu_data.pos_code,
    select_block= 1,
    select_pos= -1,
    event ="sample_on",
    time_before = time_before,
    error_type= 0,
)
#conv_out = firing_rate.convolve_signal(arr=sp_sample_on_out,fs = 1000,w_size=0.1,w_std=0.015,axis=1)
sp_sample_on_out=sp_sample_on_out[:,idx_start:idx_end]

### select neuron RF:
* 1: compare whether activity during sample or delay (in condition) is significant and larger compared to the baseline
* 2: if significant -> in, if not -> check if out condition is significant
* 3: if significant -> out, if not -> in

In [7]:

def check_epochs_activity(signal:np.ndarray,dur_bl:int,st_1:int,end_1:int,st_2:int,end_2:int):
    if signal.shape[0] > min_trials:
        mean_1 = signal[:, dur_bl + st_1 : dur_bl + end_1].mean(axis=1)
        mean_2 = signal[:, dur_bl + st_2 : dur_bl + end_2].mean(axis=1)
        mean_bl = signal[:, :dur_bl].mean(axis=1)

        larger_1 = mean_bl.mean() < mean_1.mean()
        larger_2 = mean_bl.mean() < mean_2.mean()

        # paired sample t-test: compare baseline vs epochs
        p_1 = stats.ttest_rel(mean_bl, mean_1)[1]
        p_2 = stats.ttest_rel(mean_bl, mean_2)[1]
    else:
        larger_1,larger_2 = False,False
        p_1,p_2 = np.nan, np.nan
    return larger_1,larger_2,p_1,p_2


In [9]:
v_larger_in,d_larger_in,p_v_in,p_d_in = check_epochs_activity(sp_sample_on_in,dur_fix,st_v,end_v,st_d,end_d)
v_larger_out,d_larger_out,p_v_out,p_d_out = check_epochs_activity(sp_sample_on_out,dur_fix,st_v,end_v,st_d,end_d)

In [10]:
check_in = np.logical_and(np.logical_or(v_larger_in,d_larger_in),np.logical_or(p_v_in<p_threshold,p_d_in<p_threshold))
check_out = np.logical_and(np.logical_or(v_larger_out,d_larger_out),np.logical_or(p_v_out<p_threshold,p_d_out<p_threshold))
if check_in:
    rf = 'in'
    significant = True
elif check_out:
    rf='out'
    significant = True
else:
    rf='in'
    significant = False
    

#### Compute VD index

In [12]:
if rf == 'in':
    sp = sp_sample_on_in
elif rf == 'out':
    sp = sp_sample_on_out
if significant:
    mean_visual = sp[:, dur_fix + st_v : dur_fix + end_v].mean()
    mean_delay = sp[:, dur_fix + st_d : dur_fix + end_d].mean()
    max_fr = sp[:,  : dur_fix + end_d].mean()
    mean_delay = mean_delay / max_fr  
    mean_visual = mean_visual / max_fr  
    vd_index = (mean_delay - mean_visual) / (mean_delay + mean_visual)
else:
    vd_index=np.nan

In [13]:
vd_index

-0.18008142812402123

In [ ]:
# neu_data.vd_index=vd_index
# neu_data.to_python_hdf5('ddd.h5')
# dd = NeuronData.from_python_hdf5('ddd.h5')